In [256]:
import numpy as np
import pandas as pd
import torch
import sys
import pickle

from sklearn.manifold import TSNE
import plotly.graph_objects as go

from src.models.scale import Scale
from src.visualization.scale_color_filter import get_scale_triads_filter

# Loading Model and Vocabulary

In [257]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torch.load(f"result_model/cbow_network.pt", map_location=device)

with open('./data/training_data.pkl', 'rb') as file:
        data = pickle.load(file, encoding="latin1")

vocab = data['chords_vocabulary']

c:\Users\Proprio\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\serialization.py:779: UserWarning:

'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)



# Getting Embeddings

In [258]:
# embedding from first model layer
embeddings = list(model.parameters())[0]
embeddings = embeddings.cpu().detach().numpy()

# normalization
norms = (embeddings ** 2).sum(axis=1) ** (1 / 2)
norms = np.reshape(norms, (len(norms), 1))
embeddings_norm = embeddings / norms
embeddings_norm.shape

(298, 25)

# Visualization with t-SNE

In [259]:
# get embeddings
embeddings_df = pd.DataFrame(embeddings)

# t-SNE transform
tsne = TSNE(n_components=2)
embeddings_df_trans = tsne.fit_transform(embeddings_df)
embeddings_df_trans = pd.DataFrame(embeddings_df_trans)

# get token order
chord_suffixes = np.array(list(vocab.suffixes_to_indexes.keys()))
embeddings_df_trans.index = chord_suffixes

In [272]:
scale = Scale('C', 'major')
color = get_scale_triads_filter(chord_suffixes, scale)

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=embeddings_df_trans[0],
        y=embeddings_df_trans[1],
        mode="text",
        text=embeddings_df_trans.index,
        textposition="middle center",
        textfont=dict(color=color)
    )
)

# Similare chords

In [273]:
def get_top_similar(chord_suffix: str, topN: int = 10):
    chord_id = vocab.suffixes_to_indexes[chord_suffix]

    if chord_id == 0:
        print("Out of vocabulary chord")
        return

    chord_vec = embeddings_norm[chord_id]
    chord_vec = np.reshape(chord_vec, (len(chord_vec), 1))
    dists = np.matmul(embeddings_norm, chord_vec).flatten()
    topN_ids = np.argsort(-dists)[1 : topN + 1]

    topN_dict = {}
    for sim_chord_id in topN_ids:
        sim_chord = vocab.indexes_to_chords[sim_chord_id]
        sim_chord_suffix = ''.join(sorted(sim_chord.note_suffixes))
        topN_dict[sim_chord_suffix] = dists[sim_chord_id]

    return topN_dict

In [274]:
for chord, sim in get_top_similar("CEG").items():
    print("{}: {:.3f}".format(chord, sim))

BDG: 0.736
CDG: 0.735
CDE: 0.685
DEG: 0.684
BCG: 0.633
ACE: 0.625
ADF: 0.608
ABF: 0.595
CEF: 0.580
ACF: 0.573
